In [3]:
import sys
sys.path += ['./Code']
from collections import Counter
import supernumber as sn
from supernumber import SuperNumber  

# CS2006 P1: Classes and iterators in Python

Authors: 190006203, 200010611, 200024484  
Tutor: Edwin Brady  
Date: 12 Mar 2021

## 1. Overview

For this practical we were asked to work as a team of Research Software Engineers that had been contacted by a mathematician. The topic of research the mathematician was interested in was supernumbers. Therefore, we had to create a supernumbers implementation with all of the necessary functions and properties. We then had to set up automated testing to enable the mathematician to check certain properties. We implemented the basic requirements and all extensions. We took care to document and test our code exhaustively, achieving 100% test coverage and 100% documentation coverage excluding nested functions and unit tests.



## 2. Design & Implementation

Our implementation is split into several files:
* `core.py` provides the core implementation of the `SuperNumber` and `SuperNumbers` classes.
* `props.py` and `props_supernumbers.py` provide two equivalent implementations of several functions that determine properties of a set of supernumbers. `props_checked.py` has the same interface, but uses both implementations and checks the results.
* `span.py` implements the set span algorithm.
* `supernumber.py` exports everything from one file for convenient usage by the researcher.

### `SuperNumber` and `SuperNumbers`

The `SuperNumber` and `SuperNumbers` classes are fairly straightforward implementations of the classes specified in the basic requirements and first Medium extension of the spec, respectively. A few notes on our implementation decisions:

* The spec leaves open the question of how the `SuperNumber` and `SuperNumbers` constructors should handle numbers or multipliers larger than the modulus value. We decided to simply silently apply the modulus; this simplifies the implementation of `SuperRootsOfOne`, whose definition depends on `<1 mod 1>` being equivalent to `<0 mod 1>`.
* In addition to the required methods, `SuperNumbers` implements a method, `iter_below`, that takes a supernumber and returns an iterator of every member of the set whose underlying number is less than the argument. This allows us to do something similar to the `for x in range(10) for y in range(x)` pattern, used in several of the functions in `props.py`, in `props_supernumbers.py`.

### Properties

We provided two different implementations of `HasSuperNumberIdempotentProperty`, `IsCommutativeSuperNumberMultiplication`, `IsAssociativeSuperNumberMultiplication`, and `SuperRootsOfOne`. The first implementation, in `props.py`, uses `SuperNumber` directly; the second, in `props_supernumbers.py`, uses the `SuperNumbers` abstraction. Finally, `props_checked.py` has the same interface as the other two files, but calls both implementations and asserts that the result is equal before returning it. This final module is the one used to generate the results shown in this report.

### Set Spans

`SuperNumberSetSpan`, given a set of supernumbers (the "generators", no relation to the Python language feature), finds the set of supernumbers that can be obtained by multiplying together the generators in various combinations. The implementation works by taking the initial set of generators (which are trivially part of the span), multiplying each known result by each of the generators, and repeating until the set of known results stops growing.

This implementation assumes supernumber multiplication is commutative. If it were not, we would need to test multiplying supernumbers together both ways, complicating our implementation slightly. Luckily, based on our preliminary findings, this doesn't seem to be something we have to worry about. Nevertheless, as we haven't formally proven that this is the case, `SuperNumberSetSpan` includes an assertion to ensure that this is the case—and, therefore, we can return a correct result—for the given n and alpha.

## 3. Findings

### Idempotence

Using `HasSuperNumberIdempotentProperty`, it is possible to determine whether supernumber multiplication is idempotent. This appears to be the case for some, but not all, combinations of multiplier and modulus. 

In [4]:
sn.HasSuperNumberIdempotentProperty(1, 0)

True

In [5]:
sn.HasSuperNumberIdempotentProperty(2, 0)

False

In [6]:
sn.HasSuperNumberIdempotentProperty(2, 1)

True

### Commutativity

`IsCommutativeSuperNumberMultipliction` allows us to test whether multiplication is commutative for a given set of super numbers. This is the case for all supernumbers with 1  ≤ n ≤ 50 and 0 ≤ alpha ≤ n.

In [7]:
sets = 0
commutative_sets = 0

for n in range(1,51):
    for alpha in range(0,n):
        sets += 1
        if sn.IsCommutativeSuperNumberMultiplication(n,alpha):
            commutative_sets += 1
        else:
            print(f"Multiplication is not commutative for n={n}, alpha={alpha}")

print(f"{commutative_sets}/{sets} (n, alpha) combinations are commutative")

1275/1275 (n, alpha) combinations are commutative


### Associativity

`IsAssociativeSuperNumberMultiplication` tests whether multiplication of a given set of supernumbers is associative. This is the case for all supernumbers where 1 ≤ n ≤ 20 and 0 ≤ alpha ≤ n.

In [8]:
sets = 0
associative_sets = 0

for n in range(1,21):
    for alpha in range(0,n):
        sets += 1
        if sn.IsAssociativeSuperNumberMultiplication(n,alpha):
            associative_sets += 1
        else:
            print(f"Multiplication is not associative for n={n}, alpha={alpha}")

print(f"{associative_sets}/{sets} (n, alpha) combinations are associative")

210/210 (n, alpha) combinations are associative


### Superroots of One

`SuperRootsOfOne` returns the set of supernumbers with the given `n` and `alpha` that, when multiplied by themselves, return 1.

For the supernumbers where 1 ≤ n ≤ 25, most (n, alpha) combinations result in no superroots of 1, quite a few have either 1 or 2, and a few have up to 5.

In [9]:
# Counts how many times as given number of superroots shows up
superroots_counter = Counter()
for n in range(1, 26):
    for alpha in range(0, n):
        root_count = len(sn.SuperRootsOfOne(n, alpha))
        superroots_counter[root_count] += 1
sorted([x for x in superroots_counter.items()])

[(0, 208), (1, 45), (2, 66), (3, 2), (4, 3), (5, 1)]

## 4. Testing

We implemented a suite of unit tests in `Code/UnitTests.py`, containing a total of 45 tests, our code with 100% coverage. For the two implementations of props, we use a base class, `PropsTests`, with a set of test cases, and subclass it to test the two concrete implementations. There's a bit of trickery in making this work: if we made `PropsTests` a subclass of `TestCase`, unittest would treat it as a test case in its own right and try to execute it. To get around this, the base class doesn't descend from `TestCase`; instead, the two concrete classes use multiple inheritance to descend from both `PropsTests` and `TestCase`.

Additionally, `props_checked` verifies that results are consistent across the two implementations of `props`, so any end-user usage of `props_checked` (including this report) serves as some additional testing.

All of the properties tests are run using at least one trivial case and one non-trivial case. There is a proof provided for all of the trivial cases in the comments above the trivial case to show that the properties functions are correct. For the commutative property, the trivial case uses a mod of 2 and an alpha of 0 because can only differentiate between the ordering of 2 values if they are different values. Using a mod of 2 means there are two possible values: 1 and 0 but using a mod of 1 only has the value 0 so you can't differentiate between 0 * 0 and 0 * 0. All of the other properties use the trivial case of mod = 1 and alpha = 0 as you don't need to differentiate between two different values. For example, with the `test_associative` function which is testing the `IsAssociativeSuperNumberMultiplication` property the trivial case used is mod = 1 and alpha = 0 and the non-trivial case used is mod = 4 and alpha = 5.

Because we have tested every function and property using multiple cases, we are confident our solution works as described in the practical.

## 5. Evaluation

This practical was very different to anything we have completed before as we have never written research software or had experience working with research software. The hardest part of the practical was trying to interpret the mathematics behind the properties. However, we did complete all of the basic and extension requirements so this shows that the practical was still a success.

Testing code that calculates a novel result presents some challenges—while we can ensure the results are correct by performing the calculations by hand, the lack of knowledge about the problem domain means we are unsure where the edge cases might be.

Something we noticed in our code is that the `IsAssociativeSuperNumberMultiplication` property runs slowly when the number gets big. This is because it has complexity $O(n^2)$. However, unfortunately we couldn't think of a way to make it any faster as it has to go through all of the pairs which is of $O(n^2)$ complexity. On the other hand, something that is good about our practical is our investigation. Moving our report to a Jupyter Notebook made the investigation so much more clear and reproducible.

## 6. Per-file Summary and Provenance

* `supernumber.py` - This is contains the `SuperNumber` and `SuperNumbers` (iterator) class. The SuperNumber class is an extension from the original code given in the specification and the SuperNumbers class was written purely by us.
* `props.py` - This contains all of the properties for supernumbers that we were asked to investigate, using the SuperNumber class. This was written by us.
* `props_supernumbers.py` - This contains all of the properties for supernumbers but uses the SuperNumbers iterator class. This was written by us.
* `props_checked.py` - This calls all of the properties functions for both the `SuperNumber` and `SuperNumbers` implementation, checks that the result is equal and throws an exception if they are not. This was written by us.
* `span.py` - This file calculates set spans. It was written entirely by us.
* `unit_tests.py` - This is all of the unit tests for the `SuperNumber`, `SuperNumbers` classes and the properties. This was all completely original, except the testing for both implementation of props was taken from https://stackoverflow.com/a/49545467
* `test-coverage.sh` - This shell script runs the unit tests with coverage insturmentation, and opens a browser to the results. It contains a snippet from https://stackoverflow.com/a/677212, but was otherwise written by us.
* `test-coverage.sh` - This shell script calculates documentation coverage for the project. It contains the same snippet from the script above; the remaining one line was written by us.

## 7. Conclusion
This practical went very well overall. None of us had experience working with research software or doing research software engineering so it was good to get some experience in this field. Had we have had more time, it would've been good to extend the investigation to more cases and have several investigations into each property.

## 8. Bibliography

1. Konoval, R (2018). How to use same unit test for different implementations in Python, [Online] stackoverflow.com Available at: https://stackoverflow.com/a/49545467 (last accessed: 12/03/2021)  
2. lhunath (2009). How can I check if a program exists from a Bash script?, [Online] stackoverflow.com Available at: https://stackoverflow.com/a/677212 (last accessed: 12/03/2021)